Libraries:

In [5]:
import warnings
warnings.filterwarnings('ignore')
from nltk import tokenize

from nltk.stem.wordnet import WordNetLemmatizer
from googlesearch import search 

from bs4 import BeautifulSoup
from bs4.element import Comment
import urllib.request

import wikipediaapi
import concurrent.futures
from tqdm import tqdm

import itertools
import collections
import textrank
import textract

from textblob import TextBlob
from nltk.corpus import stopwords 
from heapq import nlargest

import urllib
import requests
import time
import random

import re
import logging
import numpy as np
import pandas as pd
import nltk

from requests import get
from pattern.web import plaintext

from wordcloud import WordCloud
import matplotlib.pyplot as plt

from pprint import pprint as print
from gensim.summarization import summarize
from gensim.summarization import keywords
from gensim.summarization import mz_keywords

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.cluster.util import cosine_distance

import networkx as nx
from collections import OrderedDict
import trkeyword

Disable logging:

In [6]:
import logging, sys
logging.disable(sys.maxsize)

Load parameters:

In [65]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\skamenshchikov\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\skamenshchikov\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

CDFs:

In [8]:
def wiki_scrape(topic_name, verbose=True):
    def wiki_link(link):
        try:
            page = wiki_api.page(link)
            if page.exists():
                d = {'page': link, 'text': page.text, 'link': page.fullurl,
                     'categories': list(page.categories.keys())}
                return d
        except:
            return None

    wiki_api = wikipediaapi.Wikipedia(language='en',
        extract_format=wikipediaapi.ExtractFormat.WIKI)
    page_name = wiki_api.page(topic_name)
    if not page_name.exists():
        print('page {} does not exist'.format(topic_name))
        return
    page_links = list(page_name.links.keys())
    progress = tqdm(desc='Links Scraped', unit='', total=len(page_links)) if verbose else None
    sources = [{'page': topic_name, 'text': page_name.text, 'link': page_name.fullurl,
                'categories': list(page_name.categories.keys())}]
    with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
        future_link = {executor.submit(wiki_link, link): link for link in page_links}
        for future in concurrent.futures.as_completed(future_link):
            data = future.result()
            progress.update(1) if verbose else None
            if data:
                sources.append(data)
    progress.close() if verbose else None
    blacklist = ('Template', 'Help:', 'Category:', 'Portal:', 'Wikipedia:', 'Talk:')
    sources = pd.DataFrame(sources)
    sources = sources[(len(sources['text']) > 20)
                      & ~(sources['page'].str.startswith(blacklist))]
    sources['categories'] = sources.categories.apply(lambda x: [y[9:] for y in x])
    sources['topic'] = topic_name
    return sources

def wiki_page(page_name):
    wiki_api = wikipediaapi.Wikipedia(language='en', extract_format=wikipediaapi.ExtractFormat.WIKI)
    page_name = wiki_api.page(page_name)
    if not page_name.exists():
        print('page does not exist')
        return
    page_data = {'page': page_name, 'text': page_name.text, 'link': page_name.fullurl,
                 'categories': [[y[9:] for y in list(page_name.categories.keys())]]}
    page_data_df = pd.DataFrame(page_data)
    return page_data_df

class TextRank4Keyword():
    def __init__(self):
        self.d = 0.85 # damping coefficient, usually is .85
        self.min_diff = 1e-5 # convergence threshold
        self.steps = 10 # iteration steps
        self.node_weight = None # save keywords and its weight

    
    def set_stopwords(self, stopwords):  
        """Set stop words"""
        for word in STOP_WORDS.union(set(stopwords)):
            lexeme = nlp.vocab[word]
            lexeme.is_stop = True
    
    def sentence_segment(self, doc, candidate_pos, lower):
        """Store those words only in cadidate_pos"""
        sentences = []
        for sent in doc.sents:
            selected_words = []
            for token in sent:
                # Store words only with cadidate POS tag
                if token.pos_ in candidate_pos and token.is_stop is False:
                    if lower is True:
                        selected_words.append(token.text.lower())
                    else:
                        selected_words.append(token.text)
            sentences.append(selected_words)
        return sentences
        
    def get_vocab(self, sentences):
        """Get all tokens"""
        vocab = OrderedDict()
        i = 0
        for sentence in sentences:
            for word in sentence:
                if word not in vocab:
                    vocab[word] = i
                    i += 1
        return vocab
    
    def get_token_pairs(self, window_size, sentences):
        """Build token_pairs from windows in sentences"""
        token_pairs = list()
        for sentence in sentences:
            for i, word in enumerate(sentence):
                for j in range(i+1, i+window_size):
                    if j >= len(sentence):
                        break
                    pair = (word, sentence[j])
                    if pair not in token_pairs:
                        token_pairs.append(pair)
        return token_pairs
        
    def symmetrize(self, a):
        return a + a.T - np.diag(a.diagonal())
    
    def get_matrix(self, vocab, token_pairs):
        """Get normalized matrix"""
        # Build matrix
        vocab_size = len(vocab)
        g = np.zeros((vocab_size, vocab_size), dtype='float')
        for word1, word2 in token_pairs:
            i, j = vocab[word1], vocab[word2]
            g[i][j] = 1
            
        # Get Symmeric matrix
        g = self.symmetrize(g)
        
        # Normalize matrix by column
        norm = np.sum(g, axis=0)
        g_norm = np.divide(g, norm, where=norm!=0) # this is ignore the 0 element in norm
        
        return g_norm

    
    def get_keywords(self, number=10):
        """Print top number keywords"""
        node_weight = OrderedDict(sorted(self.node_weight.items(), key=lambda t: t[1], reverse=True))
        for i, (key, value) in enumerate(node_weight.items()):
            print(key + ' - ' + str(round(value,1)))
            if i > number:
                break
        
        
    def analyze(self, text, 
                candidate_pos=['NOUN', 'PROPN'], 
                window_size=4, lower=False, stopwords=list()):
        """Main function to analyze text"""
        
        # Set stop words
        self.set_stopwords(stopwords)
        
        # Pare text by spaCy
        doc = nlp(text)
        
        # Filter sentences
        sentences = self.sentence_segment(doc, candidate_pos, lower) # list of list of words
        
        # Build vocabulary
        vocab = self.get_vocab(sentences)
        
        # Get token_pairs from windows
        token_pairs = self.get_token_pairs(window_size, sentences)
        
        # Get normalized matrix
        g = self.get_matrix(vocab, token_pairs)
        
        # Initionlization for weight(pagerank value)
        pr = np.array([1] * len(vocab))
        
        # Iteration
        previous_pr = 0
        for epoch in range(self.steps):
            pr = (1-self.d) + self.d * np.dot(g, pr)
            if abs(previous_pr - sum(pr))  < self.min_diff:
                break
            else:
                previous_pr = sum(pr)

        # Get weight for each node
        node_weight = dict()
        for word, index in vocab.items():
            node_weight[word] = pr[index]
        
        self.node_weight = node_weight
        
def text_summarizer(raw_docx):
    
    raw_text = raw_docx
    docx = nlp(raw_text)
    stopwords = list(STOP_WORDS)

    word_frequencies = {}  
    for word in tqdm(docx):  
        if word.text not in stopwords:
            if word.text not in word_frequencies.keys():
                word_frequencies[word.text] = 1
            else:
                word_frequencies[word.text] += 1


    maximum_frequncy = max(word_frequencies.values())

    for word in tqdm(word_frequencies.keys()):  
        word_frequencies[word] = (word_frequencies[word]/maximum_frequncy)

    sentence_list = [ sentence for sentence in docx.sents ]

    # Calculate Sentence Score and Ranking
    sentence_scores = {}  
    for sent in tqdm(sentence_list):  
        for word in sent:
            if word.text.lower() in word_frequencies.keys():
                if len(sent.text.split(' ')) < 30:
                    if sent not in sentence_scores.keys():
                        sentence_scores[sent] = word_frequencies[word.text.lower()]
                    else:
                        sentence_scores[sent] += word_frequencies[word.text.lower()]

    summary_sentences = nlargest(7, sentence_scores, key=sentence_scores.get)
    final_sentences = [ w.text for w in summary_sentences ]
    summary = ' '.join(final_sentences)

    print(summary)
    
def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True

def text_from_html(body):
    soup = BeautifulSoup(body, 'html.parser')
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)  
    return u" ".join(t.strip() for t in visible_texts)

### Keyword extraction

Import libraries:

In [ ]:
from requests import get
from nltk.tokenize import word_tokenize

def tags(x):
    return nltk.pos_tag(nltk.word_tokenize(x))[0][1]

def text_from_html(body):
    soup = BeautifulSoup(body, 'html.parser')
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)  
    return u" ".join(t.strip() for t in visible_texts)

def parse_page(url):
    htmlString = get(url).text

    soup = BeautifulSoup(htmlString, 'html.parser')
    paragraphs = soup.find_all(['p', 'article', 'section', 'title', 'h1', 'h2', 'h3'])
    txt = text_from_html(str(paragraphs))

    title = soup.find('title')
    title_str = text_normalize(str(title.string).split('|', 1)[0])

    keywords = ','.join([text_from_html(str(i)) for i in soup.find_all('a') if '/tagged' in str(i)]) 
    return keywords 

List of pages:

In [ ]:
pages_list = pd.read_excel(f'datasets/urls.xlsx', sheet_name='Sheet1')
# pages_list = list(pages_list['url'])

Extract from web pages:

In [ ]:
keys_list = []
web_list = []

for i in tqdm(pages_list):
    keys_list.append(parse_page(i))
    web_list.append(i)

Write to dataframe:

In [ ]:
df = pd.DataFrame(list(zip(web_list, keys_list)), columns=['url','keys'])
df.to_excel (r'keys.xlsx', index = False, header=True)
df.head()

### Web page extraction:

Extract from document:

In [ ]:
# docxtxt = textract.process('docs/test.docx').decode('utf-8')
# docxtxt = re.sub('[^A-Za-z0-9]+', ' ', docxtxt)

Extract from Google Search:

In [ ]:
# txt = []

# exc_number = 0
# agg_number = 0

# query = 'DeepFaceDrawing'
# url_list = list(search(query, num=100, stop=100, pause=1))

# for j in tqdm(url_list):
    
#     agg_number += 1   
#     if agg_number==11:
#         break
    
#     try: 
#         delta = random.randint(1,8)
#         time.sleep(delta) 
#         html = urllib.request.urlopen(j).read()
#         txt.append(text_from_html(html))
#     except:
#         agg_number -= 1
#         exc_number += 1
        
# summ = ''.join(txt)
# webText = re.sub('[^A-Za-z0-9.]+', ' ', summ)

### POS Tagging - DT

Create random seed:

In [120]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline

from nltk import word_tokenize, pos_tag
from nltk.corpus import treebank
import random
import random
import nltk

Upload datasets:

In [121]:
nltk.download('treebank')
tagged_sentences = treebank.tagged_sents(tagset='universal')

print('Tagged sentences: ', len(tagged_sentences)) 
print('Tagged words:', len(nltk.corpus.treebank.tagged_words())) 

[nltk_data] Downloading package treebank to
[nltk_data]     C:\Users\skamenshchikov\AppData\Roaming\nltk_data...
[nltk_data]   Package treebank is already up-to-date!


Tagged sentences:  3914
Tagged words: 100676


Show random sentence:

In [122]:
print(random.choice(tagged_sentences))

[('Among', 'ADP'), ('its', 'PRON'), ('new', 'ADJ'), ('customers', 'NOUN'), (':', '.'), ('day-care', 'ADJ'), ('centers', 'NOUN'), ('that', 'DET'), ('*T*-1', 'X'), ('previously', 'ADV'), ('spurned', 'VERB'), ('the', 'DET'), ('service', 'NOUN'), ('.', '.')]


CDFs:

In [123]:
def untag(tagged_sentence):
    return [w for w, t in tagged_sentence]

def pos_tag(sentence): 
    tags = classifier.predict([features(sentence, index) for index in range(len(sentence))]) 
    return tags

def transform_to_dataset(tagged_sentences):  
    X, y = [], []      
    for tagged in tagged_sentences:         
        for index in range(len(tagged)):    
            X.append(features(untag(tagged), index))              
            y.append(tagged[index][1])       
    return X, y

def features(sentence_terms, index):
    term = sentence_terms[index]
    return {
        'nb_terms': len(sentence_terms),
        'term': term,
        'is_first': index == 0,
        'is_last': index == len(sentence_terms) - 1,
        'is_capitalized': term[0].upper() == term[0],
        'is_all_caps': term.upper() == term,
        'is_all_lower': term.lower() == term,
        'prefix-1': term[0],
        'prefix-2': term[:2],
        'prefix-3': term[:3],
        'suffix-1': term[-1],
        'suffix-2': term[-2:],
        'suffix-3': term[-3:],
        'prev_word': '' if index == 0 else sentence_terms[index - 1],
        'next_word': '' if index == len(sentence_terms) - 1 else sentence_terms[index + 1]}

Create train/test split:

In [124]:
part=int(.75 * len(tagged_sentences)) 
training_sentences = tagged_sentences[:part] 
test_sentences = tagged_sentences[part:]        
X, y = transform_to_dataset(training_sentences)

Vectorize datasets:

In [125]:
dict_vectorizer = DictVectorizer(sparse=False)
X, y = transform_to_dataset(training_sentences)
z = dict_vectorizer.fit(X,y)

Build pipeline:

In [126]:
basic  = DecisionTreeClassifier(criterion='entropy')
model = Pipeline([('vectorizer', DictVectorizer(sparse=False)), ('classifier', basic)], verbose=True)  
model.fit(X[:10000], y[:10000])

[Pipeline] ........ (step 1 of 2) Processing vectorizer, total=   0.5s
[Pipeline] ........ (step 2 of 2) Processing classifier, total=  32.2s


Pipeline(memory=None,
         steps=[('vectorizer',
                 DictVectorizer(dtype=<class 'numpy.float64'>, separator='=',
                                sort=True, sparse=False)),
                ('classifier',
                 DecisionTreeClassifier(class_weight=None, criterion='entropy',
                                        max_depth=None, max_features=None,
                                        max_leaf_nodes=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=1, min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        presort=False, random_state=None,
                                        splitter='best'))],
         verbose=True)

Score pipeline in test part:

In [127]:
X_test, y_test = transform_to_dataset(test_sentences)
print('Accuracy:', round(model.score(X_test, y_test),2))

Accuracy: 0.91


Score pipeline in cross validaion:

In [128]:
kfold = KFold(n_splits=10, random_state=7)
results = cross_val_score(model, X[:5000], y[:5000], cv=kfold)
print('Mean accuracy:', round(results.mean(),2))
print('Accuracy dispersion:', round(results.std(),2))

[Pipeline] ........ (step 1 of 2) Processing vectorizer, total=   0.2s
[Pipeline] ........ (step 2 of 2) Processing classifier, total=   4.5s
[Pipeline] ........ (step 1 of 2) Processing vectorizer, total=   0.2s
[Pipeline] ........ (step 2 of 2) Processing classifier, total=   4.2s
[Pipeline] ........ (step 1 of 2) Processing vectorizer, total=   0.2s
[Pipeline] ........ (step 2 of 2) Processing classifier, total=   4.4s
[Pipeline] ........ (step 1 of 2) Processing vectorizer, total=   0.2s
[Pipeline] ........ (step 2 of 2) Processing classifier, total=   4.2s
[Pipeline] ........ (step 1 of 2) Processing vectorizer, total=   0.2s
[Pipeline] ........ (step 2 of 2) Processing classifier, total=   4.3s
[Pipeline] ........ (step 1 of 2) Processing vectorizer, total=   0.2s
[Pipeline] ........ (step 2 of 2) Processing classifier, total=   4.4s
[Pipeline] ........ (step 1 of 2) Processing vectorizer, total=   0.2s
[Pipeline] ........ (step 2 of 2) Processing classifier, total=   4.7s
[Pipel

Apply model:

In [129]:
print(pos_tag(word_tokenize('I left my religion')))

['PRON' 'VERB' 'ADJ' 'NOUN']


### POS tagging - CRF

Load libraries and sets:

In [9]:
import nltk
from nltk.tag.util import untag
from sklearn_crfsuite import CRF
from sklearn_crfsuite import metrics
 
tagged_sentences = nltk.corpus.treebank.tagged_sents()

CDFs:

In [48]:
def pos_tag(sentence):
    sentence_features = [features(sentence, index) for index in range(len(sentence))]
    return list(zip(sentence, model.predict([sentence_features])[0]))

def transform_to_dataset(tagged_sentences):
    X, y = [], []
 
    for tagged in tagged_sentences:
        X.append([features(untag(tagged), index) for index in range(len(tagged))])
        y.append([tag for _, tag in tagged])
 
    return X, y

def features(sentence, index):
    """ sentence: [w1, w2, ...], index: the index of the word """
    return {
        'word': sentence[index],
        'is_first': index == 0,
        'is_last': index == len(sentence) - 1,
        'is_capitalized': sentence[index][0].upper() == sentence[index][0],
        'is_all_caps': sentence[index].upper() == sentence[index],
        'is_all_lower': sentence[index].lower() == sentence[index],
        'prefix-1': sentence[index][0],
        'prefix-2': sentence[index][:2],
        'prefix-3': sentence[index][:3],
        'suffix-1': sentence[index][-1],
        'suffix-2': sentence[index][-2:],
        'suffix-3': sentence[index][-3:],
        'prev_word': '' if index == 0 else sentence[index - 1],
        'next_word': '' if index == len(sentence) - 1 else sentence[index + 1],
        'has_hyphen': '-' in sentence[index],
        'is_numeric': sentence[index].isdigit(),
        'capitals_inside': sentence[index][1:].lower() != sentence[index][1:]
    }

Split into train and test:

In [150]:
cutoff = int(.75 * len(tagged_sentences))
training_sentences = tagged_sentences[:cutoff]
test_sentences = tagged_sentences[cutoff:]
 
X_train, y_train = transform_to_dataset(training_sentences)
X_test, y_test = transform_to_dataset(test_sentences)

Fit the model:

In [156]:
model = CRF()
z = model.fit(X_train, y_train)

Check the metrics:

In [159]:
y_pred = model.predict(X_test)
print('Accuracy:', round(metrics.flat_accuracy_score(y_test, y_pred),2))

Accuracy: 0.96


Predict the sentence:

In [152]:
sentence = ['I', 'am', 'Bob','!']
print(pos_tag(sentence))

[('I', 'PRP'), ('am', 'VBP'), ('Bob', 'NNP'), ('!', '.')]


### POS Tagging - LSTM

In [31]:
import nltk
from keras import backend as K
 
tagged_sentences = nltk.corpus.treebank.tagged_sents()

In [40]:
def logits_to_tokens(sequences, index):
    token_sequences = []
    for categorical_sequence in sequences:
        token_sequence = []
        for categorical in categorical_sequence:
            token_sequence.append(index[np.argmax(categorical)])
 
        token_sequences.append(token_sequence)
 
    return token_sequences

def to_categorical(sequences, categories):
    cat_sequences = []
    for s in sequences:
        cats = []
        for item in s:
            cats.append(np.zeros(categories))
            cats[-1][item] = 1.0
        cat_sequences.append(cats)
    return np.array(cat_sequences)

In [32]:
import numpy as np
sentences, sentence_tags =[], [] 

for tagged_sentence in tagged_sentences:
    sentence, tags = zip(*tagged_sentence)
    sentences.append(np.array(sentence))

In [34]:
sentences, sentence_tags =[], [] 

for tagged_sentence in tagged_sentences:
    sentence, tags = zip(*tagged_sentence)
    sentences.append(np.array(sentence))
    sentence_tags.append(np.array(tags))
 
# Let's see how a sequence looks
 
print(sentences[5])
print(sentence_tags[5])

array(['Lorillard', 'Inc.', ',', 'the', 'unit', 'of', 'New', 'York-based',
       'Loews', 'Corp.', 'that', '*T*-2', 'makes', 'Kent', 'cigarettes',
       ',', 'stopped', 'using', 'crocidolite', 'in', 'its', 'Micronite',
       'cigarette', 'filters', 'in', '1956', '.'], dtype='<U11')
array(['NNP', 'NNP', ',', 'DT', 'NN', 'IN', 'JJ', 'JJ', 'NNP', 'NNP',
       'WDT', '-NONE-', 'VBZ', 'NNP', 'NNS', ',', 'VBD', 'VBG', 'NN',
       'IN', 'PRP$', 'NN', 'NN', 'NNS', 'IN', 'CD', '.'], dtype='<U6')


In [35]:
from sklearn.model_selection import train_test_split
(train_sentences, test_sentences, train_tags, test_tags) = train_test_split(sentences, sentence_tags, test_size=0.2)

In [36]:
words, tags = set([]), set([])
 
for s in train_sentences:
    for w in s:
        words.add(w.lower())
 
for ts in train_tags:
    for t in ts:
        tags.add(t)
 
word2index = {w: i + 2 for i, w in enumerate(list(words))}
word2index['-PAD-'] = 0  # The special value used for padding
word2index['-OOV-'] = 1  # The special value used for OOVs
 
tag2index = {t: i + 1 for i, t in enumerate(list(tags))}
tag2index['-PAD-'] = 0  # The special value used to padding

In [37]:
train_sentences_X, test_sentences_X, train_tags_y, test_tags_y = [], [], [], []
 
for s in train_sentences:
    s_int = []
    for w in s:
        try:
            s_int.append(word2index[w.lower()])
        except KeyError:
            s_int.append(word2index['-OOV-'])
 
    train_sentences_X.append(s_int)
 
for s in test_sentences:
    s_int = []
    for w in s:
        try:
            s_int.append(word2index[w.lower()])
        except KeyError:
            s_int.append(word2index['-OOV-'])
 
    test_sentences_X.append(s_int)
 
for s in train_tags:
    train_tags_y.append([tag2index[t] for t in s])
 
for s in test_tags:
    test_tags_y.append([tag2index[t] for t in s])
    
def ignore_class_accuracy(to_ignore=0):
    def ignore_accuracy(y_true, y_pred):
        y_true_class = K.argmax(y_true, axis=-1)
        y_pred_class = K.argmax(y_pred, axis=-1)
 
        ignore_mask = K.cast(K.not_equal(y_pred_class, to_ignore), 'int32')
        matches = K.cast(K.equal(y_true_class, y_pred_class), 'int32') * ignore_mask
        accuracy = K.sum(matches) / K.maximum(K.sum(ignore_mask), 1)
        return accuracy
    return ignore_accuracy    

In [38]:
from keras.preprocessing.sequence import pad_sequences
MAX_LENGTH = len(max(train_sentences_X, key=len))
 
train_sentences_X = pad_sequences(train_sentences_X, maxlen=MAX_LENGTH, padding='post')
test_sentences_X = pad_sequences(test_sentences_X, maxlen=MAX_LENGTH, padding='post')
train_tags_y = pad_sequences(train_tags_y, maxlen=MAX_LENGTH, padding='post')
test_tags_y = pad_sequences(test_tags_y, maxlen=MAX_LENGTH, padding='post')

In [39]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, InputLayer, Bidirectional, TimeDistributed, Embedding, Activation
from keras.optimizers import Adam
from keras import backend as K
 
model = Sequential()
model.add(InputLayer(input_shape=(MAX_LENGTH, )))
model.add(Embedding(len(word2index), 128))
model.add(Bidirectional(LSTM(256, return_sequences=True)))
model.add(TimeDistributed(Dense(len(tag2index))))
model.add(Activation('softmax'))
 
model.compile(loss='categorical_crossentropy', optimizer=Adam(0.001), metrics=['accuracy', ignore_class_accuracy(0)])
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 271, 128)          1306496   
_________________________________________________________________
bidirectional_4 (Bidirection (None, 271, 512)          788480    
_________________________________________________________________
time_distributed_4 (TimeDist (None, 271, 47)           24111     
_________________________________________________________________
activation_4 (Activation)    (None, 271, 47)           0         
Total params: 2,119,087
Trainable params: 2,119,087
Non-trainable params: 0
_________________________________________________________________


In [41]:
cat_train_tags_y = to_categorical(train_tags_y, len(tag2index))

In [42]:
model.fit(train_sentences_X, to_categorical(train_tags_y, len(tag2index)), batch_size=128, epochs=40, validation_split=0.2)

Train on 2504 samples, validate on 627 samples
Epoch 1/40
2504/2504 [==============================] - 128s 51ms/step - loss: 1.2172 - accuracy: 0.8577 - ignore_accuracy: 0.0172 - val_loss: 0.3565 - val_accuracy: 0.9078 - val_ignore_accuracy: 0.1176
Epoch 2/40
2504/2504 [==============================] - 118s 47ms/step - loss: 0.3307 - accuracy: 0.9070 - ignore_accuracy: 0.0651 - val_loss: 0.3185 - val_accuracy: 0.9049 - val_ignore_accuracy: 0.0000e+00
Epoch 3/40
2504/2504 [==============================] - 119s 47ms/step - loss: 0.3137 - accuracy: 0.9105 - ignore_accuracy: 0.1217 - val_loss: 0.3070 - val_accuracy: 0.9172 - val_ignore_accuracy: 0.1373
Epoch 4/40
2504/2504 [==============================] - 119s 48ms/step - loss: 0.3038 - accuracy: 0.9165 - ignore_accuracy: 0.1337 - val_loss: 0.2993 - val_accuracy: 0.9175 - val_ignore_accuracy: 0.1391
Epoch 5/40
2504/2504 [==============================] - 118s 47ms/step - loss: 0.2959 - accuracy: 0.9169 - ignore_accuracy: 0.1353 - val_

Test set evaluation:

In [43]:
scores = model.evaluate(test_sentences_X, to_categorical(test_tags_y, len(tag2index)))
print(f'{model.metrics_names[1]}: {scores[1] * 100}')

783/783 [==============================] - 20s 25ms/step
'accuracy: 99.08385276794434'


Save model:

In [64]:
model.save('models/my_model.h5')